## MLflow's Model Registry

In [2]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [5]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [6]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [7]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [8]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 828b70f8080d4fa0b0c5e64b570a849d, rmse: 6.0016
run id: 3d27404f24ad486089b6146a81a6fa03, rmse: 6.1135
run id: 49df1818ef804bdfbb990a341e17050a, rmse: 6.1751
run id: 7a2a7dc509f444edaac28740c1034c33, rmse: 6.3919
run id: 647c721d549141efa6b082c7174d44b1, rmse: 6.5090


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [30]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [11]:
run_id = "647c721d549141efa6b082c7174d44b1"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/27 16:31:59 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685205119017, current_stage='None', description=None, last_updated_timestamp=1685205119017, name='nyc-taxi-regressor', run_id='647c721d549141efa6b082c7174d44b1', run_link=None, source='/home/isalimov/study/mlops-zoomcamp/02-experiment-tracking/mlruns/1/647c721d549141efa6b082c7174d44b1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [37]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Archived
version: 2, stage: Production
version: 3, stage: Staging


In [36]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685201904359, current_stage='Archived', description='', last_updated_timestamp=1685207282314, name='nyc-taxi-regressor', run_id='828b70f8080d4fa0b0c5e64b570a849d', run_link='', source='/home/isalimov/study/mlops-zoomcamp/02-experiment-tracking/mlruns/1/828b70f8080d4fa0b0c5e64b570a849d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [14]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685205119017, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-05-27', last_updated_timestamp=1685205548987, name='nyc-taxi-regressor', run_id='647c721d549141efa6b082c7174d44b1', run_link=None, source='/home/isalimov/study/mlops-zoomcamp/02-experiment-tracking/mlruns/1/647c721d549141efa6b082c7174d44b1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [17]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [19]:
df = read_dataframe("../01-intro/data/yellow_tripdata_2022-01.parquet")
df = df.iloc[10000:20000]

In [20]:
client.download_artifacts(run_id="3d27404f24ad486089b6146a81a6fa03", path='preprocessor', dst_path='.')

/tmp/ipykernel_2817049/286638808.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id="3d27404f24ad486089b6146a81a6fa03", path='preprocessor', dst_path='.')


'/home/isalimov/study/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [21]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [22]:
X_test = preprocess(df, dv)

In [23]:
target = "duration"
y_test = df[target].values

In [38]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/05/27 17:08:23 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


[17:08:23] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 882 ms, sys: 57.8 ms, total: 940 ms
Wall time: 75.3 ms


{'rmse': 4.618872778174256}

In [39]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

2023/05/27 17:08:26 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 566 ms, sys: 107 ms, total: 673 ms
Wall time: 671 ms


{'rmse': 4.666040178290721}

In [2]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)